# Orchestration Project Setup

This notebook outlines the process of setting up Orchestration dependencies for the Conversational Assistant demo.

It goes through the following steps:
- Importing an Orchestration project based on example data.
- Training an Orchestration model.
- Deploying a trained Orchestration model for production use.

**Note**: these steps can also be achieved using Azure AI [Language Studio](https://language.cognitive.azure.com/).

The example data is in the context of a fictional outdoor company: Contoso Outdoors. The project will be linked to existing CLU and CQA projects in this context.

To use Orchestration in this project, the following lines in your `.env` file should be updated by the end of this notebook:
```
ORCHESTRATION_PROJECT_NAME="<orchestration-project-name>"
ORCHESTRATION_DEPLOYMENT_NAME="<orchestration-deployment-name>"
```

### Prerequisites:
Orchestration requires an [Azure AI Language resource](https://learn.microsoft.com/en-us/azure/ai-services/language-service/). [Create a Language Resource](https://portal.azure.com/#create/Microsoft.CognitiveServicesTextAnalytics) first, if necessary.

From your Language resource, obtain its `endpoint` and `api-key`. Further, you should have an existing CLU and CQA project/deployment in this Language resource. This can be achieved by going through `clu_setup.ipynb` and `cqa_setup.ipynb`.

Update your `.env` file:
```
LANGUAGE_ENDPOINT="<language-endpoint>"
LANGUAGE_API_KEY="<language-api-key>"

CLU_PROJECT_NAME="<clu-project-name>"
CLU_DEPLOYMENT_NAME="<clu-deployment-name>"

CQA_PROJECT_NAME="<cqa-project-name>"
CQA_DEPLOYMENT_NAME="<cqa-deployment-name>"
```

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
import sys
sys.path.append("..")
import os
from dotenv import load_dotenv
from clu_client import CluClient as OrchestrationClient

load_dotenv()

LANGUAGE_ENDPOINT = os.environ.get("LANGUAGE_ENDPOINT")
LANGUAGE_API_KEY = os.environ.get("LANGUAGE_API_KEY")

# Update with desired names:
ORCHESTRATION_PROJECT_NAME = "contoso-outdoors-orch"
ORCHESTRATION_MODEL_NAME = "contoso-outdoors-orch-model-1"
ORCHESTRATION_DEPLOYMENT_NAME = "contoso-outdoors-orch-dep-1"

# Linked CLU/CQA info:
CLU_PROJECT_NAME = os.environ.get("CLU_PROJECT_NAME")
CLU_DEPLOYMENT_NAME = os.environ.get("CLU_DEPLOYMENT_NAME")
CQA_PROJECT_NAME = os.environ.get("CQA_PROJECT_NAME")

ORCHESTRATION_CLIENT = OrchestrationClient(
    endpoint=LANGUAGE_ENDPOINT,
    api_key=LANGUAGE_API_KEY,
    orchestration=True
)

print(f"Language endpoint: {LANGUAGE_ENDPOINT}")


Import Project Data:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
import json

import_file = "data/orchestration/import.json"
with open(import_file, 'r') as fp:
    import_json = json.load(fp)

import_json["metadata"]["projectName"] = ORCHESTRATION_PROJECT_NAME
print(f"Project name: {ORCHESTRATION_PROJECT_NAME}")

# Link CLU/CQA projects:
clu_intent = import_json["assets"]["intents"][0]
cqa_intent = import_json["assets"]["intents"][1]
clu_intent["orchestration"]["conversationOrchestration"]["projectName"] = CLU_PROJECT_NAME
clu_intent["orchestration"]["conversationOrchestration"]["deploymentName"] = CLU_DEPLOYMENT_NAME
cqa_intent["orchestration"]["questionAnsweringOrchestration"]["projectName"] = CQA_PROJECT_NAME

import_result = ORCHESTRATION_CLIENT.import_project(
    import_json=import_json,
    project_name=ORCHESTRATION_PROJECT_NAME
)

print(f"Import project {'succeeded' if import_result else 'failed'}")


List Projects:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print("Orchestration projects:")
projects = ORCHESTRATION_CLIENT.list_projects()
print(projects)


Train Model:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print(f"Model name: {ORCHESTRATION_MODEL_NAME}")
train_job_url = ORCHESTRATION_CLIENT.train(
    project_name=ORCHESTRATION_PROJECT_NAME,
    model_name=ORCHESTRATION_MODEL_NAME
)
print("Model training in progress...")


Check Training Progress:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print(f"Training progress for model: {ORCHESTRATION_MODEL_NAME}")
train_job_status = ORCHESTRATION_CLIENT.get_training_status(
    train_job_url=train_job_url
)
print(train_job_status)
print(f"Training percent complete: {train_job_status['result']['trainingStatus']['percentComplete']}")
print(f"Evaluation percent complete: {train_job_status['result']['evaluationStatus']['percentComplete']}")
print(f"Model training: {train_job_status['status']}")


List Trained Models:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print("Orchestration models:")
models = ORCHESTRATION_CLIENT.list_models(
    project_name=ORCHESTRATION_PROJECT_NAME
)
print(models)


Deploy Model:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print(f"Creating deployment: {ORCHESTRATION_DEPLOYMENT_NAME}")
deploy_result = ORCHESTRATION_CLIENT.deploy(
    project_name=ORCHESTRATION_PROJECT_NAME,
    model_name=ORCHESTRATION_MODEL_NAME,
    deployment_name=ORCHESTRATION_DEPLOYMENT_NAME
)
print(f"Deployment {'succeeded' if deploy_result else 'failed'}")


List Deployments:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
print("CLU deployments:")
deployments = ORCHESTRATION_CLIENT.list_deployments(
    project_name=ORCHESTRATION_PROJECT_NAME
)
print(deployments)


Test Deployment:

In [ ]:
# Copyright (c) Microsoft Corporation.
# Licensed under the MIT License.
from language_http_utils import run_language_api

url = "".join([
    LANGUAGE_ENDPOINT,
    "/language/:analyze-conversations"
])
query_parameters = {
    "api-version": "2023-04-01"
}

def create_input(
    utterance: str,
    language: str,
    id: str
) -> dict:
    """
    Create JSON input for Orchestration runtime.
    """
    return {
        "kind": "Conversation",
        "analysisInput": {
            "conversationItem": {
                "id": str(id),
                "participantId": "0",
                "language": language,
                "text": utterance
            }
        },
        "parameters": {
            "projectName": ORCHESTRATION_PROJECT_NAME,
            "deploymentName": ORCHESTRATION_DEPLOYMENT_NAME
        }
    }

def call_runtime(
    utterance: str,
    language: str,
    id: str
) -> dict:
    """
    Call Orchestration runtime.
    """
    input_json = create_input(
        utterance=utterance,
        language=language,
        id=id
    )

    return run_language_api(
        url=url,
        api_key=LANGUAGE_API_KEY,
        json_obj=input_json,
        query_parameters=query_parameters
    )

# Update query to test:
query = "What is the order status for order 12?"

clu_response = call_runtime(
    utterance=query,
    language="en",
    id="1"
)

print(f"Orchestration Runtime Response:\n{clu_response}")


## Next steps:
Update your `.env` file:
```
ORCHESTRATION_PROJECT_NAME="<orchestration-project-name>"
ORCHESTRATION_DEPLOYMENT_NAME="<orchestration-deployment-name>"
```

You may experiment updating your Orchestration project in [Language Studio](https://language.cognitive.azure.com/) as well.